# Generating Image Overlays of Wx Variables

First obtain the maximum and minimum values of Latitude and Longitude so that the image can be placed in Leaflet.

In [13]:
lat_min = float(lats.min())
lat_max = float(lats.max())
lon_min = float(lons.min())
lon_max = float(lons.max())

print(f"Image bounds: SW=({lat_min}, {lon_min}), NE=({lat_max}, {lon_max})")


Image bounds: SW=(-3.236724853515625, 44.73521041870117), NE=(43.468475341796875, 111.26478576660156)


Lets generate the images.

## Rainfall

In [39]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from tqdm import notebook


# Load WRF data and Calculate Wind Speed
ds = xr.open_dataset("/Users/manaruchi/Desktop/WeatherDataViz/raw_data/AFCNWP_WRF_model_output_00UTC.nc")

# Generate hourly images for 3 days
tlimit = 71

for t in notebook.tqdm(range(tlimit), desc="Generating Hourly Rainfall Images.."):
    rainc1 = ds["RAINC"].isel(Time=t)
    rainc2 = ds["RAINC"].isel(Time=t+1)
    rainnc1 = ds["RAINNC"].isel(Time=t)
    rainnc2 = ds["RAINNC"].isel(Time=t+1)
    lats = ds["XLAT"].isel(Time=t).values
    lons = ds["XLONG"].isel(Time=t).values
    
    rain_actual = (rainc2 - rainc1) + (rainnc2 - rainnc1)
    
    # Smooth Temperature with Gaussian filter (sigma controls smoothness)
    rain_smooth = gaussian_filter(rain_actual, sigma=1)
    
    # For rainfall, using green as color
    greens = [
      "#464444", # Dark
      "#f7fbff",  # Very light blue (almost white)
      "#deebf7",  # Pale blue
      "#c6dbef",  # Soft blue
      "#9ecae1",  # Sky blue
      "#6baed6",  # Medium light blue
      "#4292c6",  # Medium blue
      "#2171b5",  # Deep sky blue
      "#08519c",  # Strong blue
      "#08306b",  # Dark navy blue
      "#041f4a",  # Very dark blue
      "#021025",   # Near-black blue
      "#fddbc7",  # Light Pink
      "#f4a582",  # Peach
      "#d6604d",  # Light Red
      "#b2182b",  # Red
      "#67001f"   # Dark Red
    ]
    
    
    rain_cmap = LinearSegmentedColormap.from_list("windy", greens)
    # rain_cmap.set_bad(color='none')
    
    # Mask 0s
    # rain_smooth = np.ma.masked_where(rain_smooth < 0.1, rain_smooth)
    
    # Plot
    fig = plt.figure(figsize=(20, 16))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax.axis("off")
    ax.set_frame_on(False)
    fig.patch.set_alpha(0.0)
    
    # Compute image extent for imshow: [left, right, bottom, top]
    extent = [lons.min(), lons.max(), lats.min(), lats.max()]
    
    # Use imshow with interpolation for smooth result
    img = ax.imshow(
        rain_smooth,
        origin='lower',
        extent=extent,
        cmap=rain_cmap,
        vmin=0,
        vmax=30,
        interpolation='bilinear',   # 'bilinear' or 'bicubic' for smoothness
        alpha=0.8
    )
    
    ax.set_extent(extent, crs=ccrs.PlateCarree())
    
    plt.savefig(
        f"rainfall_{t}_0.png",
        dpi=300,
        bbox_inches='tight',
        pad_inches=0,
        transparent=True
    )
    plt.close()

Generating Hourly Rainfall Images..:   0%|          | 0/71 [00:00<?, ?it/s]